In [12]:
import json  # Handles JSON data
import re  # Provides regular expression matching
import google.auth  # Handles Google authentication
from googleapiclient.discovery import build  # Google API client for service discovery
from googleapiclient.errors import HttpError  # Handles errors from Google API calls
from google.colab import auth  # Google Colab authentication module
from markdown import markdown  # Converts Markdown text to HTML
import unittest  # Python unit testing framework

In [13]:
def authenticate_google_docs():
    """Authenticates with Google Docs API and returns a service object."""
    auth.authenticate_user()
    creds, _ = google.auth.default()
    return build('docs', 'v1', credentials=creds)

def create_google_doc(service, title):
    """Creates a new Google Doc with the given title and returns its document ID."""
    doc = service.documents().create(body={'title': title}).execute()
    return doc['documentId']

def update_google_doc(service, doc_id, requests):
    '''Updates the specified Google Doc with the given list of requests.'''
    try:
        service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()  # Apply updates
        print(f'Document updated successfully: https://docs.google.com/document/d/{doc_id}')
    except HttpError as e:
        print(f'An error occurred: {e}')  # Handle API errors

In [19]:
def parse_markdown(md_text):
    """Converts markdown text into Google Docs API requests with proper bullet formatting and indentation."""
    requests = []
    index = 1  # Google Docs uses 1-based indexing
    lines = md_text.split("\n")  # Split input into lines

    for line in lines:
        actual_text = line.rstrip()
        skipped_chars = 0
        style = None
        apply_footer_style = False
        bullet = False

        # Handling other markdown elements
        if line.startswith("# "):  # Heading 1
            actual_text = line[2:]
            skipped_chars = 2
            style = "HEADING_1"

        elif line.startswith("## "):  # Heading 2
            actual_text = line[3:]
            skipped_chars = 3
            style = "HEADING_2"

        elif line.startswith("### "):  # Heading 3
            actual_text = line[4:]
            skipped_chars = 4
            style = "HEADING_3"

        elif line.lower().startswith("meeting recorded by") or line.lower().startswith("duration"):
            apply_footer_style = True  # Footer styling flag

        # Insert text normally
        actual_text_length = len(actual_text) + 1  # +1 for newline
        requests.append({
            "insertText": {
                "location": {"index": index},
                "text": actual_text + "\n"
            }
        })

        # Apply heading styles if needed
        if style:
            requests.append({
                "updateParagraphStyle": {
                    "range": {"startIndex": index, "endIndex": index + actual_text_length},
                    "paragraphStyle": {"namedStyleType": style},
                    "fields": "namedStyleType"
                }
            })

        # Apply footer styling if detected
        if apply_footer_style:
            requests.append({
                "updateTextStyle": {
                    "range": {"startIndex": index, "endIndex": index + actual_text_length},
                    "textStyle": {"italic": True, "fontSize": {"magnitude": 10, "unit": "PT"}},
                    "fields": "italic,fontSize"
                }
            })

        # Apply bold to @mentions only
        for match in re.finditer(r"@\w+", actual_text):
            mention_start = index + match.start()
            mention_end = index + match.end()
            requests.append({
                "updateTextStyle": {
                    "range": {"startIndex": mention_start, "endIndex": mention_end},
                    "textStyle": {"bold": True},
                    "fields": "bold"
                }
            })

        # Update the index position
        index += actual_text_length  # Move index forward

    return requests

In [20]:
# Example markdown notes
markdown_notes = """# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

# Main execution
service = authenticate_google_docs()
doc_id = create_google_doc(service, "Product Team Sync - May 15, 2023")
requests = parse_markdown(markdown_notes)
update_google_doc(service, doc_id, requests)


Document updated successfully: https://docs.google.com/document/d/1aruN4DkZG4eWW1Lq0neimfRcQdIuXMPEkmF0kBhv5Z4
